# Likwala-aux-Herbes-Region

In [1]:
%matplotlib inline
from IPython.display import display
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlite3

con = sqlite3.connect("../data/CongoDB.sqlite")

## Stilgruppen

Bedingungen:
* nur Funde aus dem Likwala-Gebiet
* Typen sind klar und enthalten keine '?' und sind nicht leer

In [10]:
sql = """SELECT t_Obj.objID,
            t_Ort.ort_name AS ort_kurz,
            t_Ort.ort_fluss,
            t_Ort."Kat-Nr",
            t_Obj.Typ
        FROM t_Komplex INNER JOIN t_Obj ON t_Komplex.komplexID = t_Obj.komplexID
            INNER JOIN t_Ort ON t_Komplex.ortID = t_Ort.ortID
        WHERE (((t_Ort.ort_lit) = 'DS')
            AND ((t_Obj.Typ) Not Like '')
            AND ((t_Obj.Typ) Not Like '%/%')
            AND ((t_Obj.Typ) Not Like '%(?)%')
            AND ((t_Ort.ort_fluss) Like '%Likwala%'))""".replace('\n',' ')

df = pd.read_sql(sql, con)

df['STIL'] = df['Typ']

stilgr = pd.read_csv("../lit/Wotzka1995_StilGrKuerzel.csv")
stilgr['STILGR'] = stilgr['Seidensticker'] + ' ' + stilgr['Stilgruppe']

# dictionary der Stilgruppen-Kürzel und Stilgruppen
di = stilgr.set_index('Typ').to_dict()['STILGR']

df = df.replace({"Typ": di})

a = df['Typ'].str.split(' ', expand = True)
b = a[0].str.split('.', expand = True)

b[1] = pd.to_numeric(b[1])
b[2] = pd.to_numeric(b[2])
b[3] = pd.to_numeric(b[3])

df = pd.concat([df, b], axis = 1)

df = df.replace('', np.nan, regex=True)
sicher = df[pd.notnull(df['Typ'])]

sicher.head()

,objID,ort_kurz,ort_fluss,Kat-Nr,Typ,STIL,0,1,2,3
0,70,Boyenge,Likwala-Esobe,284,4.3.1.4 Botendo,BOT,4,3.0,1.0,4.0
1,163,Itanga,Likwala-Esobe,305,4.2.2.6 Epena,EPE,4,2.0,2.0,6.0
2,205,Likwala-Esobe km 186,Likwala-Esobe,291,LKW,LKW,LKW,NaN,NaN,NaN
3,310,Mosengi,Likwala-Esobe,296,4.2.2.6 Epena,EPE,4,2.0,2.0,6.0
4,329,Munda,Likwala-Esobe,304,4.2.2.5 Ebambe,EBA,4,2.0,2.0,5.0


In [11]:
df_pivot = df.pivot_table(values = 'objID', index = ['Kat-Nr', 'ort_kurz'], columns = [1, 2, 3, 'STIL'], aggfunc = len)
df_pivot = df_pivot.sort_index(ascending=[False])
display(df_pivot.head())

1                              2.0                                  3.0
2                              2.0                                  1.0
3                             1.0  3.0  4.0   5.0   6.0  7.0  13.0 4.0 
STIL                           PKM  NGO  MAT   EBA   EPE  MKA  BBS  BOT
Kat-Nr ort_kurz                                                        
306    Epena                   NaN  NaN  NaN   NaN   2.0  NaN  NaN  NaN
305    Itanga                  NaN  NaN  NaN   3.0  23.0  2.0  NaN  NaN
304    Munda                 113.0  NaN  NaN  91.0  18.0  NaN  NaN  NaN
303    Jeke                    NaN  NaN  NaN  10.0  19.0  3.0  NaN  NaN
301    Likwala-Esobe km 401    NaN  NaN  1.0   NaN   NaN  NaN  NaN  NaN

In [12]:
# Tabelle nur mit den fraglichen Stücken

sql = """SELECT t_Obj.objID,
            t_Ort.ort_name AS ort_kurz,
            t_Ort.ort_fluss,
            t_Ort."Kat-Nr",
            t_Obj.Typ
        FROM t_Komplex INNER JOIN t_Obj ON t_Komplex.komplexID = t_Obj.komplexID
            INNER JOIN t_Ort ON t_Komplex.ortID = t_Ort.ortID
        WHERE (((t_Ort.ort_lit) = 'DS')
            AND ((t_Obj.Typ) Like '%/%')
            OR ((t_Obj.Typ) Like '%(?)%')
            AND ((t_Ort.ort_fluss) Like '%Likwala%'))""".replace('\n',' ')

df = pd.read_sql(sql, con)

df['Kat-Nr'] = pd.to_numeric(df['Kat-Nr'])

df = df[df['Kat-Nr'] > 282]
# (?) und alles nach / weg
df['Typ'] = df['Typ'].map(lambda x: x.rstrip(' (?)'))
df['Typ'] = df['Typ'].apply(lambda x: x.split('/')[0])

df['STIL'] = df['Typ']

df = df.replace({"Typ": di})

a = df['Typ'].str.split(' ', expand = True)
b = a[0].str.split('.', expand = True)

b[1] = pd.to_numeric(b[1])
b[2] = pd.to_numeric(b[2])
b[3] = pd.to_numeric(b[3])

df = pd.concat([df, b], axis = 1)

df_pivot = df.pivot_table(values = 'objID', index = 'Kat-Nr', columns = ['Typ', 'STIL'], aggfunc = len, fill_value=0)
df_pivot = df_pivot.sort_index(ascending=[False])

fragl = df_pivot

fragl[fragl != 0] = 0.5
fragl = fragl.unstack().reset_index()
fragl.head()

,Typ,STIL,Kat-Nr,0
0,4.2.2.1 Pikunda-Munda,PKM,283,0.0
1,4.2.2.1 Pikunda-Munda,PKM,284,0.0
2,4.2.2.1 Pikunda-Munda,PKM,285,0.5
3,4.2.2.1 Pikunda-Munda,PKM,286,0.0
4,4.2.2.1 Pikunda-Munda,PKM,287,0.0


In [13]:
len(fragl)

220

In [14]:
df_pivot = sicher.pivot_table(values = 'objID', index = 'Kat-Nr', columns = ['Typ', 'STIL'], aggfunc = len, fill_value=0)
df_pivot = df_pivot.sort_index(ascending=[False])
sicherPiv = df_pivot

sicherPiv[sicherPiv != 0] = 1
sicherPiv = sicherPiv.unstack().reset_index()
sicherPiv.head()

,Typ,STIL,Kat-Nr,0
0,4.2.2.1 Pikunda-Munda,PKM,283,0
1,4.2.2.1 Pikunda-Munda,PKM,284,0
2,4.2.2.1 Pikunda-Munda,PKM,285,0
3,4.2.2.1 Pikunda-Munda,PKM,286,0
4,4.2.2.1 Pikunda-Munda,PKM,287,0


In [15]:
len(sicherPiv)

189

In [16]:
styles = sicherPiv.append(fragl, ignore_index = True)
styles['Kat-Nr'] = pd.to_numeric(styles['Kat-Nr'])
styles = styles.rename(columns={0:'Val'})
styles.head()

,Typ,STIL,Kat-Nr,Val
0,4.2.2.1 Pikunda-Munda,PKM,283,0.0
1,4.2.2.1 Pikunda-Munda,PKM,284,0.0
2,4.2.2.1 Pikunda-Munda,PKM,285,0.0
3,4.2.2.1 Pikunda-Munda,PKM,286,0.0
4,4.2.2.1 Pikunda-Munda,PKM,287,0.0


In [17]:
len(styles)

409

In [18]:
a = styles['Typ'].str.split(' ', expand = True)
b = a[0].str.split('.', expand = True)

b[1] = pd.to_numeric(b[1])
b[2] = pd.to_numeric(b[2])
b[3] = pd.to_numeric(b[3])

styles = pd.concat([styles, b], axis = 1)

In [19]:
orte = df.pivot_table(values = 'objID', index = ['Kat-Nr', 'ort_kurz'], aggfunc = len)
orte = orte.reset_index()

styles = pd.merge(styles, orte, on='Kat-Nr', how='left')

In [20]:
stylesPiv = styles.pivot_table(values = 'Val', index = ['Kat-Nr', 'ort_kurz'], columns = [1, 2, 3, 'STIL'], aggfunc = sum, fill_value=0)
stylesPiv = stylesPiv.sort_index(ascending=[False])
stylesPiv[stylesPiv == 0] = '-'
stylesPiv[stylesPiv == 1.5] = "$\\bullet $"
stylesPiv[stylesPiv == 1.0] = "$\\bullet $"
stylesPiv[stylesPiv == 0.5] = "$\circ $"
stylesPiv.reset_index().to_latex('../output/tabs/4-2-1_Likwala-Gebiet_Sequenz.tex',
                                 index = False, escape = False, encoding='utf-8')
stylesPiv

1                       2.0                                                \
2                       2.0                                                 
3                      1.0         3.0       4.0         5.0         6.0    
STIL                    PKM         NGO       MAT         EBA         EPE   
Kat-Nr ort_kurz                                                             
306    Epena       $\circ $           -         -           -  $\bullet $   
305    Itanga      $\circ $           -         -  $\bullet $  $\bullet $   
304    Munda     $\bullet $           -         -  $\bullet $  $\bullet $   
303    Jeke        $\circ $    $\circ $         -  $\bullet $  $\bullet $   
302    Botongo            -           -         -    $\circ $    $\circ $   
300    Likunda            -           -         -  $\bullet $  $\bullet $   
299    Mosenge     $\circ $           -         -  $\bullet $  $\bullet $   
298    Bwanela     $\circ $           -         -           -    $\circ $   
297    Ebambe      $\circ $           -         -  $\bullet $  $\bullet $   
296    Mosengi            -           -         -  $\bullet $  $\bullet $   
295    Bokuma             -    $\circ $         -           -  $\bullet $   
292    Bojenjo     $\circ $    $\circ $         -  $\bullet $  $\bullet $   
290    Lokolélé    $\circ $           -         -  $\bullet $  $\bullet $   
289    Yumba       $\circ $           -         -  $\bullet $  $\bullet $   
288    Misongo     $\circ $           -         -           -    $\circ $   
287    Ilebo              -           -         -           -  $\bullet $   
286    Botwale            -           -         -    $\circ $  $\bullet $   
285    Boleko      $\circ $  $\bullet $         -           -  $\bullet $   
284    Boyenge            -    $\circ $         -  $\bullet $           -   
283    Ngombe             -           -  $\circ $  $\bullet $  $\bullet $   

1                                                                 3.0  \
2                                                                 1.0   
3                      7.0       10.0      11.0        13.0      2.0    
STIL                    MKA       KON       PDM         BBS       BDG   
Kat-Nr ort_kurz                                                         
306    Epena              -         -         -           -         -   
305    Itanga    $\bullet $         -         -           -         -   
304    Munda              -  $\circ $         -           -         -   
303    Jeke      $\bullet $         -         -           -         -   
302    Botongo            -         -         -           -         -   
300    Likunda            -         -         -           -         -   
299    Mosenge            -         -         -           -         -   
298    Bwanela     $\circ $         -         -           -         -   
297    Ebambe    $\bullet $         -         -           -  $\circ $   
296    Mosengi            -         -         -           -         -   
295    Bokuma    $\bullet $         -         -           -         -   
292    Bojenjo   $\bullet $         -         -           -         -   
290    Lokolélé    $\circ $         -         -           -         -   
289    Yumba     $\bullet $         -         -           -         -   
288    Misongo   $\bullet $         -         -           -         -   
287    Ilebo       $\circ $         -         -           -         -   
286    Botwale   $\bullet $         -         -           -         -   
285    Boleko    $\bullet $         -         -  $\bullet $         -   
284    Boyenge            -         -         -           -         -   
283    Ngombe    $\bullet $  $\circ $  $\circ $           -         -   

1                            
2                            
3                      4.0   
STIL                    BOT  
Kat-Nr ort_kurz              
306    Epena              -  
305    Itanga             -  
304    Munda              -  
303    J